In [1]:
import numpy as np
from Server import Server

In [2]:
%load_ext Cython

In [3]:
from ga import input_wrapper
from helpers import parse
from pruner import magic

In [4]:
path = 'input/dc.in'

In [5]:
def input_wrapper(inp):
    servers = []
    for i, v in enumerate(inp):
        r, slot = v[0]
        s, cap, id = v[1]
        serv = Server(id, s, cap)
        serv.row = r
        serv.slot = slot
        servers.append(serv)
    return servers

In [6]:
R, S, U, P, M, unavaiable, servers = parse(path)
servers = input_wrapper(magic(path))

In [7]:
servers

[S[2]: size=2, capacity=40, row=7, slot=0 | P[-1],
 S[7]: size=2, capacity=40, row=7, slot=2 | P[-1],
 S[8]: size=3, capacity=60, row=2, slot=0 | P[-1],
 S[34]: size=2, capacity=40, row=15, slot=0 | P[-1],
 S[50]: size=4, capacity=80, row=14, slot=3 | P[-1],
 S[58]: size=1, capacity=20, row=13, slot=0 | P[-1],
 S[59]: size=5, capacity=100, row=12, slot=0 | P[-1],
 S[66]: size=3, capacity=60, row=6, slot=0 | P[-1],
 S[69]: size=2, capacity=40, row=13, slot=1 | P[-1],
 S[86]: size=3, capacity=60, row=11, slot=0 | P[-1],
 S[132]: size=5, capacity=100, row=9, slot=0 | P[-1],
 S[134]: size=2, capacity=40, row=7, slot=4 | P[-1],
 S[135]: size=3, capacity=60, row=1, slot=0 | P[-1],
 S[136]: size=1, capacity=20, row=15, slot=2 | P[-1],
 S[137]: size=1, capacity=20, row=8, slot=0 | P[-1],
 S[153]: size=3, capacity=60, row=5, slot=0 | P[-1],
 S[219]: size=3, capacity=60, row=4, slot=0 | P[-1],
 S[222]: size=4, capacity=80, row=2, slot=3 | P[-1],
 S[232]: size=4, capacity=80, row=0, slot=0 | P[-1

In [58]:
def initialize_population(population_size, M, P):
    return [np.random.randint(0, P, size=M) for _ in range(population_size)]

def score_population(population, capacities, rows, M, P, R):
    return [fitness(candidate, capacities, rows, M, P, R) for candidate in population]

def selection(population, scores, retain_frac=0.8, retain_random=0.05):
    retain_len = int(len(scores) * retain_frac)
    sorted_indices = np.argsort(scores)[::-1]
    population = [population[idx] for idx in sorted_indices]
    selected = population[:retain_len]
    leftovers = population[retain_len:]

    for gene in leftovers:
        if np.random.rand() < retain_random:
            selected.append(gene)
    return selected

def mutate(candidate, M, P, mutate_frac=0.1):
    frac = int(M * mutate_frac)
    if frac == 0:
        return
    n_tries = np.random.randint(0, int(len(candidate) * mutate_frac))

    idxs = np.random.randint(0, len(candidate), size=n_tries)
    pools = np.random.randint(0, P, size=n_tries)
    for i in range(n_tries):
        candidate[idxs[i]] = pools[i]

def crossover(mom, dad):
    select_mask = np.random.binomial(1, 0.5, size=len(mom)).astype('bool')
    child1, child2 = np.copy(mom), np.copy(dad)
    child1[select_mask] = dad[select_mask]
    child2[select_mask] = mom[select_mask]
    return child1, child2

def evolve(population, capacities, rows, M, P, R, retain_frac=0.8, retain_random=0.05, mutate_chance=0.05):
    """
    Evolution step
    :param population: list or candidate solutions
    :param target: 20x20 array that represents field in stopping condition
    :param delta: number of steps to revert
    :param retain_frac: percent of top individuals to proceed into the next genetation
    :param retain_random: chance of retaining sub-optimal individual
    :param mutate_chance: chance of mutating the particular individual
    :return: new generation of the same size
    """
    scores = score_population(population, capacities, rows, M, P, R)
    next_population = selection(population, scores, retain_frac=retain_frac, retain_random=retain_random)
    
    # mutate everyone expecting for the best candidate
    for gene in next_population[1:]:
        if np.random.random() < mutate_chance:
            mutate(gene, M, P)

    places_left = len(population) - len(next_population)
    children = []
    parent_max_idx = len(next_population) - 1
    while len(children) < places_left:
        mom_idx, dad_idx = np.random.randint(0, parent_max_idx, 2)
        if mom_idx != dad_idx:
            child1, child2 = crossover(next_population[mom_idx], next_population[dad_idx])
            children.append(child1)
            if len(children) < places_left:
                children.append(child2)
    next_population.extend(children)
    return next_population

def solve(servers, capacities, rows, M, P, R, population_size=10, n_generations=100, init_population=None):
    if init_population is None:
        population = initialize_population(population_size, M, P)
    else:
        population = init_population
    for generation in range(n_generations):
        population = evolve(population, capacities, rows, M, P, R)
        if generation == 0:
            print("Generation #: best score")
        elif generation % 200 == 0:
            print("Generation ", generation, ": ", fitness(population[0], capacities, rows, M, P, R))
    return population[0]

In [59]:
%%cython -a
import numpy as np
cimport numpy as np
from Server import Server

cpdef int fitness(np.ndarray[long, ndim=1] pools, np.ndarray[long, ndim=1] capacities, np.ndarray[long, ndim=1] rows, int M, int P, int R):
    cdef:
        int least_gc = 1000
        np.int32_t i, j, gci, cap, r, strike_cap
    for i in range(P):
        gci = 1000
        cap = 0
        for j in range(M):
            if pools[j] == i:
                cap += capacities[j]
        for r in range(R):
            strike_cap = 0
            for j in range(M):
                if pools[j] == i and rows[j] == r:
                    strike_cap += capacities[j]
            if gci > (cap - strike_cap):
                gci = cap - strike_cap
        if gci < least_gc:
            least_gc = gci
    return least_gc

In [60]:
servs = [serv.copy() for serv in servers]
capacities = np.array([int(serv.capacity) for serv in servers])
rows = np.array([serv.row for serv in servers])

In [11]:
result = solve(servs, capacities, rows, len(servers), P, R, n_generations=30000)

Generation #: best score
Generation  200 :  215
Generation  400 :  248
Generation  600 :  248
Generation  800 :  248
Generation  1000 :  248
Generation  1200 :  251
Generation  1400 :  266
Generation  1600 :  276
Generation  1800 :  280
Generation  2000 :  289
Generation  2200 :  289
Generation  2400 :  289
Generation  2600 :  289
Generation  2800 :  289
Generation  3000 :  289
Generation  3200 :  289
Generation  3400 :  289
Generation  3600 :  289
Generation  3800 :  289
Generation  4000 :  289
Generation  4200 :  289
Generation  4400 :  289
Generation  4600 :  293
Generation  4800 :  298
Generation  5000 :  298
Generation  5200 :  304
Generation  5400 :  304
Generation  5600 :  304
Generation  5800 :  304
Generation  6000 :  304
Generation  6200 :  304
Generation  6400 :  304
Generation  6600 :  304
Generation  6800 :  304
Generation  7000 :  304
Generation  7200 :  304
Generation  7400 :  304
Generation  7600 :  304
Generation  7800 :  304
Generation  8000 :  304
Generation  8200 : 

In [30]:
import scipy
import multiprocessing
import os

In [61]:

def work(servers, capacities, rows, M, P, R, population_size, n_iter):
    scipy.random.seed()
    return solve(servers, capacities, rows, M, P, R, population_size=population_size, n_generations=n_iter)

def multiprocess_solve(servers, capacities, rows, M, P, R, population_size=100, n_iter=3000):
    pool = multiprocessing.Pool(os.cpu_count())
    tasks = [([serv.copy() for serv in servers], capacities, rows, len(servers), P, R, population_size, n_iter) for _ in range(os.cpu_count())]
    results = pool.starmap(work, tasks)
    return results

In [64]:
total_results = multiprocess_solve(servs, capacities, rows, len(servers), P, R, population_size=200, n_iter=30000)

Generation #: best score
Generation #: best score
Generation #: best score
Generation #: best score
Generation  200 :  234
Generation  200 :  230
Generation  200 :  229
Generation  200 :  240
Generation  400 :  249
Generation  400 :  230
Generation  400 :  232
Generation  400 :  244
Generation  600 :  237
Generation  600 :  249
Generation  600 :  244
Generation  600 :  236
Generation  800 :  237
Generation  800 :  244
Generation  800 :  249
Generation  800 :  236
Generation  1000 :  244
Generation  1000 :  238
Generation  1000 :  249
Generation  1000 :  236
Generation  1200 :  244
Generation  1200 :  249
Generation  1200 :  244
Generation  1200 :  238
Generation  1400 :  247
Generation  1400 :  257
Generation  1400 :  244
Generation  1400 :  238
Generation  1600 :  247
Generation  1600 :  255
Generation  1600 :  257
Generation  1600 :  238
Generation  1800 :  247
Generation  1800 :  255
Generation  1800 :  257
Generation  1800 :  241
Generation  2000 :  247
Generation  2000 :  255
Gene

In [68]:
[fitness(pools, capacities, rows, len(pools), P, R) for pools in total_results]

[366, 354, 363, 364]

In [69]:
best_result = total_results[0]

In [20]:
def save_result(servers, pools, M, filename='submit.out'):
    for i in range(len(servers)):
        servers[i].pool = pools[i]
    by_id = sorted(servers, key=lambda x: x.id)
    ids = [_.id for _ in by_id]
    map = {id: serv for id, serv in zip(ids, by_id)}
#     for i in range(len(by_id)):
#         print(by_id[i])
    with open(filename, 'w+') as f:
        for i in range(M):
            if i in ids:
                print("{} {} {}".format(map[i].row, map[i].slot, map[i].pool), file=f)
            else:
                print("x", file=f)

In [70]:
save_result([s.copy() for s in servers], best_result, M, 's366.out')

S[0]: size=2, capacity=28, row=2, slot=49 | P[23]
S[1]: size=5, capacity=55, row=12, slot=70 | P[20]
S[2]: size=2, capacity=40, row=7, slot=0 | P[2]
S[4]: size=1, capacity=6, row=3, slot=65 | P[17]
S[5]: size=4, capacity=68, row=1, slot=20 | P[44]
S[7]: size=2, capacity=40, row=7, slot=2 | P[17]
S[8]: size=3, capacity=60, row=2, slot=0 | P[5]
S[10]: size=4, capacity=60, row=2, slot=38 | P[30]
S[11]: size=3, capacity=27, row=3, slot=89 | P[5]
S[13]: size=3, capacity=36, row=6, slot=63 | P[14]
S[14]: size=5, capacity=95, row=11, slot=6 | P[29]
S[15]: size=2, capacity=26, row=13, slot=53 | P[41]
S[16]: size=1, capacity=19, row=10, slot=4 | P[42]
S[17]: size=5, capacity=45, row=14, slot=86 | P[13]
S[18]: size=4, capacity=56, row=14, slot=50 | P[18]
S[19]: size=1, capacity=17, row=0, slot=9 | P[38]
S[20]: size=3, capacity=51, row=13, slot=22 | P[20]
S[23]: size=5, capacity=95, row=6, slot=12 | P[28]
S[24]: size=5, capacity=45, row=13, slot=87 | P[38]
S[25]: size=1, capacity=19, row=12, slot